In [29]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelWithLMHead
from transformers import Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import classification_report


In [2]:
print("PyTorch CUDA 버전:", torch.version.cuda)

PyTorch CUDA 버전: 12.1


In [3]:
torch.cuda.is_available()

True

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3060 Ti


In [5]:
df=pd.read_excel('천재교육 연수원 라벨링 최종본.xlsx')

In [6]:
review = df[['후기', '라벨링']]

In [7]:
review

,후기,라벨링
0,파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~감사합니다~,1
1,도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..그런데 프로...,0
2,저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...,1
3,교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...,1
4,"근래 들어 정말 기분좋게, 즐겁게 들었던 연수입니다.환경인가? 하며 들었는데 다시 ...",1
...,...,...
2893,학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창의적으로 개발하고 지속적으...,1
2894,"선생님들의 수학 개념 설명을 듣고, 실제 수학 수업을 볼 수 있어서 너무 유익했습니...",1
2895,진로라는게 흥미는 있지만 너무 포괄적이네요.,0
2896,진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.,1


In [8]:
# 결측치 제거, 라벨링 숫자형 변환
review_df = review[~review['라벨링'].str.contains('결측치').fillna(False)]
review_df['라벨링'] = pd.to_numeric(review_df['라벨링'])

C:\Users\k_373\AppData\Local\Temp\ipykernel_6076\1922987813.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['라벨링'] = pd.to_numeric(review_df['라벨링'])


In [9]:
review_df

,후기,라벨링
0,파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~감사합니다~,1
1,도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..그런데 프로...,0
2,저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...,1
3,교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...,1
4,"근래 들어 정말 기분좋게, 즐겁게 들었던 연수입니다.환경인가? 하며 들었는데 다시 ...",1
...,...,...
2893,학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창의적으로 개발하고 지속적으...,1
2894,"선생님들의 수학 개념 설명을 듣고, 실제 수학 수업을 볼 수 있어서 너무 유익했습니...",1
2895,진로라는게 흥미는 있지만 너무 포괄적이네요.,0
2896,진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.,1


In [10]:
# 전체 데이터 중 80%를 학습하고, 20%로 검증한다. 난수를 42로 주면서 재현율을 높게한다.
train_df, val_df = train_test_split(review_df, test_size=0.2, random_state=42)

In [11]:
# kcbert-large 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-large", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# 텐서를 GPU로 옮기기
tensor_on_gpu = torch.randn((100, 100)).to(device)

# 모델을 GPU로 옮기기
model_on_gpu = model.to(device)

In [13]:
# 훈련, 확인 데이터 나누고 문자열 최대길이는 300, (넘어가면 오류 남) 하고 넘어가면 자르는 방식.
train_encodings = tokenizer(train_df['후기'].tolist(), truncation=True, padding=True, max_length=300)
val_encodings = tokenizer(val_df['후기'].tolist(), truncation=True, padding=True, max_length=300)

In [14]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_df['라벨링'].tolist())
val_dataset = SentimentDataset(val_encodings, val_df['라벨링'].tolist())


In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    save_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


  1%|          | 10/1737 [00:05<13:27,  2.14it/s]

{'loss': 0.5533, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


                                                 
  1%|          | 10/1737 [00:22<13:27,  2.14it/s]

{'eval_loss': 0.4673064053058624, 'eval_runtime': 17.2778, 'eval_samples_per_second': 33.511, 'eval_steps_per_second': 8.392, 'epoch': 0.02}


  1%|          | 20/1737 [00:30<21:49,  1.31it/s]  

...

                                                     
100%|█████████▉| 1730/1737 [1:12:16<00:04,  1.46it/s]

{'eval_loss': 0.2132006585597992, 'eval_runtime': 16.7118, 'eval_samples_per_second': 34.646, 'eval_steps_per_second': 8.677, 'epoch': 2.99}


100%|██████████| 1737/1737 [1:12:21<00:00,  2.50s/it]

{'train_runtime': 4341.5926, 'train_samples_per_second': 1.598, 'train_steps_per_second': 0.4, 'train_loss': 0.2120598661430474, 'epoch': 3.0}


TrainOutput(global_step=1737, training_loss=0.2120598661430474, metrics={'train_runtime': 4341.5926, 'train_samples_per_second': 1.598, 'train_steps_per_second': 0.4, 'train_loss': 0.2120598661430474, 'epoch': 3.0})

In [16]:
trainer.save_model("./finetuned_kcbert_large")

In [21]:
model_path = "./finetuned_kcbert_large"

In [19]:
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [24]:
texts = ["유익해요", "어려워요."]

In [25]:
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model(**inputs)

In [27]:
probs = F.softmax(outputs.logits, dim=-1)
print(probs)


tensor([[1.0643e-04, 9.9989e-01],
        [9.9825e-01, 1.7488e-03]], grad_fn=<SoftmaxBackward0>)


In [28]:
predictions = torch.argmax(probs, dim=-1)
for i, text in enumerate(texts):
    print(f"Text: {text}")
    print(f"Predicted label: {predictions[i].item()}, Probability: {probs[i][predictions[i]].item()}")
    print()

Text: 유익해요
Predicted label: 1, Probability: 0.9998935461044312

Text: 어려워요.
Predicted label: 0, Probability: 0.9982511401176453



In [43]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# 모델을 평가 모드로 설정
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds = []
all_labels = []

# DataLoader 설정
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

with torch.no_grad():
    for batch in val_loader:
    # 배치에서 입력 데이터와 라벨을 분리
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)
        
        # 모델 예측
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        
        # 결과 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 성능 지표 계산 및 출력
print(classification_report(all_labels, all_preds, target_names=['Negative', 'Positive']))


              precision    recall  f1-score   support

    Negative       0.84      0.65      0.73        40
    Positive       0.97      0.99      0.98       539

    accuracy                           0.97       579
   macro avg       0.91      0.82      0.86       579
weighted avg       0.97      0.97      0.97       579



In [ ]:
### 클래스 별 성능 지표
# Negative (부정 감정)

# Precision: 84% (부정이라고 예측한 것 중 실제로 부정인 것의 비율)
# Recall: 65% (실제 부정인 것 중 모델이 부정으로 예측한 것의 비율)
# F1-score: 73% (Precision과 Recall의 조화 평균)
# Positive (긍정 감정)

# Precision: 97% (긍정이라고 예측한 것 중 실제로 긍정인 것의 비율)
# Recall: 99% (실제 긍정인 것 중 모델이 긍정으로 예측한 것의 비율)
# F1-score: 98% (Precision과 Recall의 조화 평균)
# 전체 성능 지표
# Accuracy: 97% (전체 데이터 중 모델이 올바르게 예측한 비율)
# Macro Avg:
# Precision: 91%
# Recall: 82%
# F1-score: 86%
# Weighted Avg:
# Precision: 97%
# Recall: 97%
# F1-score: 97%
# 해석
# 모델은 긍정 감정에 대해서는 매우 높은 성능을 보이지만, 부정 감정에 대해서는 상대적으로 낮은 성능을 보입니다.
# 이는 데이터셋에 긍정 감정의 샘플이 부정 감정의 샘플보다 훨씬 많기 때문일 수 있습니다 (데이터 불균형).
# 모델의 성능을 향상시키기 위해서는 부정 감정의 샘플을 추가하거나, 데이터 불균형을 해결하기 위한 방법을 고려해볼 필요가 있습니다.
# 전체적으로 모델의 성능은 매우 좋지만, 부정 감정을 더 잘 예측할 수 있도록 모델을 개선할 여지가 있습니다.